<a href="https://colab.research.google.com/github/swisscheese38/tfcatflap/blob/main/training_material/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Learn and export the model
According to https://www.tensorflow.org/lite/models/modify/model_maker/object_detection
## Install dependencies

In [1]:
!python --version
!pip --version
!pip install -q tflite-model-maker

Python 3.8.15
pip 21.1.3 from /usr/local/lib/python3.8/dist-packages/pip (python 3.8)
     |████████████████████████████████| 577 kB 29.5 MB/s 
     |████████████████████████████████| 77 kB 7.3 MB/s 
     |████████████████████████████████| 128 kB 80.0 MB/s 
     |████████████████████████████████| 1.3 MB 9.6 MB/s 
     |████████████████████████████████| 3.4 MB 10.4 MB/s 
     |████████████████████████████████| 10.9 MB 12.4 MB/s 
     |████████████████████████████████| 238 kB 81.0 MB/s 
     |████████████████████████████████| 840 kB 73.2 MB/s 
     |████████████████████████████████| 87 kB 8.0 MB/s 
     |████████████████████████████████| 1.1 MB 64.6 MB/s 
     |████████████████████████████████| 60.8 MB 1.2 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 498.0 MB 12 kB/s 
     |████████████████████████████████| 352 kB 79.9 MB/s 
     |████████████████████████████████| 5.8 MB 60.9 MB/s 
     |████████████████████████████████| 462 kB 7

## Download pictures and bounding boxes

In [3]:
#!wget https://codeload.github.com/swisscheese38/tfcatflap/tar.gz/refs/heads/main -q -O download.tgz
#!tar -zxf download.tgz --wildcards "tfcatflap-main/training_material/*.jpg" --strip-components=2
#!tar -zxf download.tgz "tfcatflap-main/training_material/index.csv" --strip-components=2
#!rm download.tgz
#!ls
!pwd

/content


## Load data


In [15]:
from __future__ import annotations
from tflite_model_maker import object_detector
from tflite_model_maker import model_spec

spec = model_spec.get('efficientdet_lite0')
train_data = object_detector.DataLoader.from_pascal_voc(images_dir='train', annotations_dir='train', label_map=['Snout'])
validation_data = object_detector.DataLoader.from_pascal_voc(images_dir='validate', annotations_dir='validate', label_map=['Snout'])
test_data = object_detector.DataLoader.from_pascal_voc(images_dir='test', annotations_dir='test', label_map=['Snout'])
#data = object_detector.DataLoader.from_pascal_voc(
#    images_dir='images', annotations_dir='labels',label_map=['Snout'])
#train_data, validation_and_test_data = data.split(0.8)
#validation_data, test_data = validation_and_test_data.split(0.5)

#train_data, validation_data, test_data = object_detector.DataLoader.from_csv('index.csv')

## Train model

In [16]:
model = object_detector.create(train_data, model_spec=spec, batch_size=4, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
6/9 [===================>..........] - ETA: 0s - det_loss: 1.7948 - cls_loss: 1.1240 - box_loss: 0.0134 - reg_l2_loss: 0.0630 - loss: 1.8578 - learning_rate: 0.0072 - gradient_norm: 2.8337

9/9 [==============================] - 41s 592ms/step - det_loss: 1.7611 - cls_loss: 1.1178 - box_loss: 0.0129 - reg_l2_loss: 0.0630 - loss: 1.8241 - learning_rate: 0.0065 - gradient_norm: 2.8153 - val_det_loss: 1.6738 - val_cls_loss: 1.0845 - val_box_loss: 0.0118 - val_reg_l2_loss: 0.0630 - val_loss: 1.7368
Epoch 2/50
9/9 [==============================] - 3s 405ms/step - det_loss: 1.6327 - cls_loss: 1.0688 - box_loss: 0.0113 - reg_l2_loss: 0.0630 - loss: 1.6957 - learning_rate: 0.0050 - gradient_norm: 3.0651 - val_det_loss: 1.5737 - val_cls_loss: 1.0203 - val_box_loss: 0.0111 - val_reg_l2_loss: 0.0630 - val_loss: 1.6367
Epoch 3/50
9/9 [==============================] - 3s 347ms/step - det_loss: 1.5283 - cls_loss: 0.9828 - box_loss: 0.0109 - reg_l2_loss: 0.0630 - loss: 1.5913 - learning_rate: 0.0050 - gradient_norm: 3.7545 - val_det_loss: 1.4387 - val_cls_loss: 0.9176 - val_box_loss: 0.0104 - val_reg_l2_loss: 0.0630 - val_loss: 1.5017
Epoch 4/50
9/9 [==============================] - 

## Test the model

In [17]:
model.evaluate(test_data)

1/1 [==============================] - 3s 3s/step



{'AP': 0.62115514,
 'AP50': 1.0,
 'AP75': 0.6442244,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.62115514,
 'ARmax1': 0.6,
 'ARmax10': 0.73333335,
 'ARmax100': 0.73333335,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.73333335,
 'AP_/Snout': 0.62115514}

## Export to tensorflow lite model

In [18]:
model.export(export_dir='.')

## Evaluate the tensorflow lite model

In [19]:
model.evaluate_tflite('model.tflite', test_data)


3/3 [==============================] - 7s 2s/step



{'AP': 0.5610561,
 'AP50': 1.0,
 'AP75': 0.44224423,
 'APs': -1.0,
 'APm': -1.0,
 'APl': 0.5610561,
 'ARmax1': 0.6,
 'ARmax10': 0.6,
 'ARmax100': 0.6,
 'ARs': -1.0,
 'ARm': -1.0,
 'ARl': 0.6,
 'AP_/Snout': 0.5610561}